In [100]:
# # reading harvardx data from the csv
# # still a work in progress since their dataset is malformatted
# import csv

# # load data
# csvfile_path = '../datasets/corpus_HarvardX_LatestCourses_based_on_2016-10-18.csv'

# with open(csvfile_path, 'r') as csvfile:
#     reader = csv.DictReader(csvfile)
#     print reader.next()

In [1]:
import search
reload(search)

# reopen index, but reload it instead of creating it afresh
udacity_se = search.UdacitySearchEngine(create=False)
    
# dictionary of results
results = udacity_se.search("neural networks")
num_results = len(results)

In [2]:
num_results

2

In [21]:
# import sys
# sys.path.append('recker')
# from recker_prototype import Recker

In [ ]:
# r = Recker

In [19]:
import gensim.models as models
import gensim.models.word2vec as word2vec

In [6]:
model = word2vec.load

AttributeError: 'module' object has no attribute 'load'

In [7]:
model = word2vec.Word2Vec.load('models/word2vec')

[(u'psychological', 0.8650830388069153),
 (u'biological', 0.8408447504043579),
 (u'behavior', 0.8405084609985352),
 (u'physiological', 0.829001247882843),
 (u'evolutionary', 0.8259677886962891),
 (u'physical', 0.8235838413238525),
 (u'theoretical', 0.8168773055076599),
 (u'therapy', 0.8051855564117432),
 (u'computational', 0.8007746934890747),
 (u'quantitative', 0.7976580262184143)]

In [ ]:

model = word2vec.Word2Vec(corpus)
model.save('models/word2vec')
most = model.most_similar(positive=['cognitive'], topn=10)

NameError: name 'corpus' is not defined

In [21]:
corpus = word2vec.Text8Corpus('datasets/enwik8')

In [29]:
ngram_phrases= models.Phrases(corpus)

In [30]:
ngram_transformer = models.phrases.Phraser(ngram_phrases)

In [ ]:
model = word2vec.Word2Vec(ngram_transformer[corpus])